<a href="https://colab.research.google.com/github/Aman-2903/Collaborative_filtering_based_recommendation_system/blob/main/course_recommendation_collabrative_filtering(Coursera).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d imuhammad/course-reviews-on-coursera

Dataset URL: https://www.kaggle.com/datasets/imuhammad/course-reviews-on-coursera
License(s): GPL-2.0
course-reviews-on-coursera.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/course-reviews-on-coursera.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
course=pd.read_csv('/content/Coursera_courses.csv')
review=pd.read_csv('/content/Coursera_reviews.csv')

In [ ]:
course.head()

,name,institution,course_url,course_id
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being
3,Technical Support Fundamentals,Google,https://www.coursera.org/learn/technical-suppo...,technical-support-fundamentals
4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,https://www.coursera.org/learn/google-cbrs-cpi...,google-cbrs-cpi-training


In [ ]:
review.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [ ]:
course.shape

(623, 4)

In [ ]:
review.shape

(1454711, 5)

In [ ]:
course.isnull().sum()

name           0
institution    0
course_url     0
course_id      0
dtype: int64

In [ ]:
review.isnull().sum()

reviews         153
reviewers         0
date_reviews      0
rating            0
course_id         0
dtype: int64

In [ ]:
review=review.dropna()

In [ ]:
review.isnull().sum()

reviews         0
reviewers       0
date_reviews    0
rating          0
course_id       0
dtype: int64

In [ ]:
course.duplicated().sum()

0

In [ ]:
review.duplicated().sum()

934672

In [ ]:
num_rating=review.groupby('course_id').count()['rating'].reset_index()
num_rating.rename(columns={'rating':'num_rating'},inplace=True)
num_rating

,course_id,num_rating
0,aboriginal-education,197
1,access-control-sscp,131
2,accounting-analytics,1356
3,accounting-data-analytics-python,24
4,actualizacion-manejo-diabetes-tipo-2,519
...,...,...
599,womens-health-human-rights,624
600,write-a-feature-length-screenplay-for-film-or-...,90
601,write-your-first-novel,52
602,writing-editing-words,309


In [ ]:
new_df=review[['course_id','rating']]
avg_rating=new_df.groupby('course_id').mean()['rating'].reset_index()
avg_rating.rename(columns={'rating':'avg_rating'},inplace=True)
avg_rating

,course_id,avg_rating
0,aboriginal-education,4.604061
1,access-control-sscp,4.572519
2,accounting-analytics,4.426991
3,accounting-data-analytics-python,3.500000
4,actualizacion-manejo-diabetes-tipo-2,4.867052
...,...,...
599,womens-health-human-rights,4.894231
600,write-a-feature-length-screenplay-for-film-or-...,4.666667
601,write-your-first-novel,3.942308
602,writing-editing-words,4.776699


In [ ]:
popular_df=num_rating.merge(avg_rating,on='course_id')
popular_df=popular_df[popular_df['num_rating']>20].sort_values('avg_rating',ascending=False).head(50)

In [ ]:
popular_df=popular_df.merge(course,on='course_id')

In [ ]:
popular_df['name']

0                           Removing Barriers to Change
1     El Abogado del Futuro: Legaltech y la Transfor...
2               Développement psychologique de l'enfant
3     The Horse Course: Introduction to Basic Care a...
4     Build a Modern Computer from First Principles:...
5                   Bugs 101: Insect-Human Interactions
6                        Economics of Money and Banking
7                    Cursos en línea: modelo para armar
8                                Everyday Excel, Part 2
9                         Equine Welfare and Management
10                                   Corporate Strategy
11                    Getting started with TensorFlow 2
12                        Programming Languages, Part A
13             In the Studio: Postwar Abstract Painting
14            More Introduction to Financial Accounting
15    Aprendiendo a aprender: Poderosas herramientas...
16    Become a CBRS Certified Professional Installer...
17                                              

In [ ]:
num_rating

,course_id,num_rating
0,aboriginal-education,197
1,access-control-sscp,131
2,accounting-analytics,1356
3,accounting-data-analytics-python,24
4,actualizacion-manejo-diabetes-tipo-2,519
...,...,...
599,womens-health-human-rights,624
600,write-a-feature-length-screenplay-for-film-or-...,90
601,write-your-first-novel,52
602,writing-editing-words,309


In [ ]:
x=review.groupby('reviewers').count()['rating']>25
padhe_likhe=x[x].index

In [ ]:
filter=review[review['reviewers'].isin(padhe_likhe)]

In [ ]:
y=filter.groupby('course_id')['rating'].count()>100
popular=y[y].index

In [ ]:
final=filter[filter['course_id'].isin(popular)]

In [ ]:
final

,reviews,reviewers,date_reviews,rating,course_id
87,This course is virtually worthless. I couldn't...,By James R,"Apr 23, 2019",1,financial-markets-global
91,I was expecting more from the course. It is ve...,By Sebastian S,"Jan 05, 2020",1,financial-markets-global
96,I find the professor is hard to understand bec...,By David S,"May 28, 2020",1,financial-markets-global
100,Not at all legitimate course. The concepts and...,By Shivam S,"May 10, 2020",1,financial-markets-global
101,"Honestly, this course could have been a lot be...",By Abhishek V,"Jul 18, 2020",1,financial-markets-global
...,...,...,...,...,...
1454664,Nice,By Daniel V,"Feb 17, 2019",5,computer-networking
1454667,Good,By Atichat P,"Dec 02, 2018",5,computer-networking
1454677,:3,By Ronaldo F N C,"Jul 14, 2020",5,computer-networking
1454689,Good,By Atichat P,"Dec 02, 2018",5,computer-networking


In [ ]:
final=final.merge(course,on='course_id')

In [ ]:
pt = final.pivot_table(index='name',columns='reviewers',values='rating')

In [ ]:
pt

reviewers,By H A H,By N B,By A S,By A S R,By A.M I H A,By ABDUL J C,By ABHIJIT S,By ABHINAV S,By ABHISHEK B,By ABHISHEK G,...,By xun y,By yan l,By yashar k s,By yaw o a,By zhe h,By Γεώργιος Κ,By Александр,By 李俊宏,By 林韋銘,By 손승건
name,,,,,,,,,,,,,,,,,,,,,
A Law Student's Toolkit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AI For Everyone,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AI for Medical Diagnosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AWS Fundamentals: Addressing Security Risk,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AWS Fundamentals: Going Cloud-Native,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Writing in the Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Étudier en France: French Intermediate course B1-B2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Математика и Python для анализа данных,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN


In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

reviewers,By H A H,By N B,By A S,By A S R,By A.M I H A,By ABDUL J C,By ABHIJIT S,By ABHINAV S,By ABHISHEK B,By ABHISHEK G,...,By xun y,By yan l,By yashar k s,By yaw o a,By zhe h,By Γεώργιος Κ,By Александр,By 李俊宏,By 林韋銘,By 손승건
name,,,,,,,,,,,,,,,,,,,,,
A Law Student's Toolkit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AI For Everyone,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AI for Medical Diagnosis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AWS Fundamentals: Addressing Security Risk,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AWS Fundamentals: Going Cloud-Native,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Writing in the Sciences,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Étudier en France: French Intermediate course B1-B2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Математика и Python для анализа данных,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
similarity_scores.shape

(390, 390)

In [ ]:
def recommend(course_name):
    # index fetch
    L=[]
    index = np.where(pt.index==course_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:4]

    data = []
    for i in similar_items:
        item = []
        temp_df = course[course['name'] == pt.index[i[0]]]
        L.append(temp_df['name'].values.tolist()[0])
        print(temp_df['name'].values)
    recommended_path.append(L)

In [ ]:
recommend('AI For Everyone')

['Neural Networks and Deep Learning']
['Programming for Everybody (Getting Started with Python)']
['Python Data Structures']
